# Assignment 2 DSC 102 2022 WI

## Introduction

In this assignment we will conduct data engineering for the Amazon dataset. It is divided into 2 parts. The extracted features in Part 1 will be used for the Part 2 of assignment, where you train a model (or models) to predict user ratings for a product.

We will be using Apache Spark for this assignment. The default Spark API will be DataFrame, as it is now the recommended choice over the RDD API. That being said, please feel free to switch back to the RDD API if you see it as a better fit for the task. We provide you an option to request RDD format to start with. Also you can switch between DataFrame and RDD in your solution. 

Another newer API is Koalas, which is also avaliable. However, it has constraints and is not applicable to most tasks. Refer to the PA statement for detail.

### Set the following parameters

In [1]:
PID = 'A16147523' # your pid, for instance: 'a43223333'
INPUT_FORMAT = 'dataframe' # choose a format of your input data, valid options: 'dataframe', 'rdd', 'koalas'

In [2]:
# Boiler plates, do NOT modify
%load_ext autoreload
%autoreload 2
import os
import getpass
from pyspark.sql import SparkSession
from utilities import SEED
from utilities import PA2Test
from utilities import PA2Data
from utilities import data_cat
from pa2_main import PA2Executor
import time
if INPUT_FORMAT == 'dataframe':
    import pyspark.ml as M
    import pyspark.sql.functions as F
    import pyspark.sql.types as T
if INPUT_FORMAT == 'koalas':
    import databricks.koalas as ks
elif INPUT_FORMAT == 'rdd':
    import pyspark.mllib as M
    from pyspark.mllib.feature import Word2Vec
    from pyspark.mllib.linalg import Vectors
    from pyspark.mllib.linalg.distributed import RowMatrix

os.environ['PYSPARK_SUBMIT_ARGS'] = '--py-files utilities.py,assignment2.py \
--deploy-mode client \
pyspark-shell'

class args:
    review_filename = data_cat.review_filename
    product_filename = data_cat.product_filename
    product_processed_filename = data_cat.product_processed_filename
    ml_features_train_filename = data_cat.ml_features_train_filename
    ml_features_test_filename = data_cat.ml_features_test_filename
    output_root = '/home/{}/{}-pa2/test_results'.format(getpass.getuser(), PID)
    test_results_root = data_cat.test_results_root
    pid = PID

pa2 = PA2Executor(args, input_format=INPUT_FORMAT)
data_io = pa2.data_io
data_dict = pa2.data_dict
begin = time.time()


Loading datasets ...Done


In [3]:
# Import your own dependencies
import pyspark.ml.feature as K
from pyspark.sql.functions import isnan
#-----------------------------

# Part 1: Feature Engineering

In [4]:
# Bring the part_1 datasets to memory and de-cache part_2 datasets. 
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_1')

# Task0: warm up 
This task is provided for you to get familiar with Spark API. We will use the dataframe API to demonstrate. Solution is given to you and this task won't be graded.

Refer to https://spark.apache.org/docs/latest/api/python/pyspark.sql.html for API guide.

The task is to implement the function below. Given the ```product_data``` table:
1. Take and print five rows.

1. Select only the ```asin``` column, then print five rows of it.

1. Select the row where ```asin = B00I8KEOTM``` and print it.

1. Count the total number of rows.

1. Calculate the mean ```price```.

1. You need to conduct the above operations, then extract some statistics out of the generated columns. You need to put the statistics in a python dictionary named ```res```. The description and schema of it are as follows:
    ```
    res
     | -- count_total: int -- count of total rows of the entire table after your operations
     | -- mean_price: float -- mean value of column price
    ```

In [5]:
def task_0(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    product_data.show(5)
    product_data[['asin']].show(5)
    product_data.where(F.col('asin') == 'B00I8KEOTM').show()
    count_rows = product_data.count()
    mean_price = product_data.select(F.avg(F.col('price'))).head()[0]
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmatically. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated with
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {'count_total': None, 'mean_price': None}
    
    # Modify res:

    res['count_total'] = count_rows
    res['mean_price'] = mean_price

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    return res
    # -------------------------------------------------------------------------

In [6]:
if INPUT_FORMAT == 'dataframe':
    res = task_0(data_io, data_dict['product'])
    pa2.tests.test(res, 'task_0')

+----------+--------------------+--------------------+--------------------+-----+--------------------+
|      asin|           salesRank|          categories|               title|price|             related|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
|B00I8HVV6E|[Home &amp; Kitch...|[[Home & Kitchen,...|Intelligent Desig...|27.99|[also_viewed -> [...|
|B00I8KEOTM|                null|[[Apps for Androi...|                null| null|[also_viewed -> [...|
|B00I8KCW4G|[Clothing -> 2233...|[[Clothing, Shoes...|eShakti Women's P...|41.95|[also_viewed -> [...|
|B00I8JKCQW|[Clothing -> 1405...|[[Clothing, Shoes...|Lady Slimming Mid...| null|[also_viewed -> [...|
|B00I8JKI8E|[Home &amp; Kitch...|[[Clothing, Shoes...|3 Tier Bangle Bra...|24.99|[also_viewed -> [...|
+----------+--------------------+--------------------+--------------------+-----+--------------------+
only showing top 5 rows

+----------+
|      asin|
+----------+
|B00I8HVV

# Task1

In [5]:
# %load -s task_1 assignment2.py
def task_1(data_io, review_data, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    overall_column = 'overall'
    # Outputs:
    mean_rating_column = 'meanRating'
    count_rating_column = 'countRating'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    grouped = review_data.groupBy('asin').agg(F.avg('overall'), F.count('overall')).toDF('asin', 'meanRating', 'countRating')
    transformed = product_data[['asin']].join(grouped, on='asin', how="left")
    count_total = transformed.count()
    # print(count_total)
    mean_meanRating = transformed.select(F.avg(F.col('meanRating'))).head()[0]
    variance_meanRating = transformed.select(F.variance(F.col('meanRating'))).head()[0]
    numNulls_meanRating = transformed.select([F.count(F.when(F.col('meanRating').isNull(), 'meanRating'))
                                              .alias('meanRating')]).head()[0]
    
    mean_countRating = transformed.select(F.avg(F.col('countRating'))).head()[0]
    variance_countRating = transformed.select(F.variance(F.col('countRating'))).head()[0]
    numNulls_countRating = transformed.select([F.count(F.when(F.col('countRating').isNull(), 'countRating'))
                                               .alias('countRating')]).head()[0]



    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    # Calculate the values programmaticly. Do not change the keys and do not
    # hard-code values in the dict. Your submission will be evaluated withVBc
    # different inputs.
    # Modify the values of the following dictionary accordingly.
    res = {
        'count_total': None,
        'mean_meanRating': None,
        'variance_meanRating': None,
        'numNulls_meanRating': None,
        'mean_countRating': None,
        'variance_countRating': None,
        'numNulls_countRating': None
    }
    # Modify res:
    res['count_total'] = count_total
    res['mean_meanRating'] = mean_meanRating
    res['variance_meanRating'] = variance_meanRating
    res['numNulls_meanRating'] = numNulls_meanRating
    res['mean_countRating'] = mean_countRating
    res['variance_countRating'] = variance_countRating
    res['numNulls_countRating'] = numNulls_countRating
    
    
    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_1')
    return res
    # -------------------------------------------------------------------------


In [36]:
res = task_1(data_io, data_dict['review'], data_dict['product'])
pa2.tests.test(res, 'task_1')

tests for task_1 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countRating ... Pass
Test 3/7 : mean_meanRating ... Pass
Test 4/7 : numNulls_countRating ... Pass
Test 5/7 : numNulls_meanRating ... Pass
Test 6/7 : variance_countRating ... Pass
Test 7/7 : variance_meanRating ... Pass
7/7 passed
-------------------------------------------------------------------------------


True


# Task 2

In [51]:
# %load -s task_2 assignment2.py
def task_2(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    salesRank_column = 'salesRank'
    categories_column = 'categories'
    asin_column = 'asin'
    # Outputs:
    category_column = 'category'
    bestSalesCategory_column = 'bestSalesCategory'
    bestSalesRank_column = 'bestSalesRank'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    def flatten_cate(x):
        if type(x) != list or len(x) == 0 or x[0][0] == "":
            return None
        return x[0][0]
    
    flatten_cate = F.udf(flatten_cate, T.StringType())
    
    cates = product_data.select(F.col('asin'), flatten_cate(F.col("categories")).alias(category_column))#.toDF('asin', category_column)
    ranks = product_data.select('asin', F.explode('salesRank')).toDF('asin', bestSalesCategory_column, bestSalesRank_column)
    transformed = cates.join(ranks, on = 'asin', how = 'left')
    
#     transformed = product_data.select('asin', F.explode('salesRank')).toDF('asin', bestSalesCategory_column, bestSalesRank_column)
#     transformed.show()
#     categories_column = product_data.select(F.col('asin'), flatten_cate(F.col("categories"))
#                                             .alias("categories")).toDF('asin', 'categories')
    
#     product_data.select('asin', F.explode('salesRank'))
    
#     def flatten_bestcate(x):
#         if type(x) != dict or len(x) == 0:
#             return None
#         return list(x.keys())[0]
    
#     def flatten_rank(x):
#         if type(x) != dict or len(x) == 0:
#             return None
#         return x[list(x.keys())[0]]
    
#     flatten_bestcate = F.udf(flatten_bestcate, T.StringType())
#     flatten_rank = F.udf(flatten_rank, T.IntegerType())
#     bestSalesCategory_column =  product_data.select(F.col('asin'), flatten_bestcate(F.col("salesRank"))
#                                                     .alias("bestSalesCategory")).toDF('asin', 'bestSalesCategory')
#     bestSalesRank_column = product_data.select(F.col('asin'), flatten_rank(F.col("salesRank"))
#                                                 .alias("bestSalesRank")).toDF('asin', 'bestSalesRank')
#     transformed = product_data[['asin']].join(categories_column, on='asin', how="left") \
#                                        .join(bestSalesCategory_column, on='asin', how="left") \
#                                         .join(bestSalesRank_column, on='asin', how="left")
    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_bestSalesRank': None,
        'variance_bestSalesRank': None,
        'numNulls_category': None,
        'countDistinct_category': None,
        'numNulls_bestSalesCategory': None,
        'countDistinct_bestSalesCategory': None
    }
    # Modify res:
    res['count_total'] = transformed.count()
    res['mean_bestSalesRank'] =  transformed.select(F.avg(F.col('bestSalesRank'))).head()[0]
    res['variance_bestSalesRank'] = transformed.select(F.variance(F.col('bestSalesRank'))).head()[0]
    res['numNulls_category'] = transformed.select([F.count(F.when(F.col(category_column).isNull(), category_column)).alias(category_column)]).head()[0]
    res['countDistinct_category'] = transformed.select(F.countDistinct(category_column)).head()[0]
    res['numNulls_bestSalesCategory'] = transformed.select([F.count(F.when(F.col('bestSalesCategory').isNull(), 'bestSalesCategory')).alias('bestSalesCategory')]).head()[0]
    res['countDistinct_bestSalesCategory'] = transformed.select(F.countDistinct('bestSalesCategory')).head()[0]
    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_2')
    return res
    # -------------------------------------------------------------------------


In [52]:
res = task_2(data_io, data_dict['product'])
pa2.tests.test(res, 'task_2')

tests for task_2 --------------------------------------------------------------
Test 1/7 : countDistinct_bestSalesCategory ... Pass
Test 2/7 : countDistinct_category ... Pass
Test 3/7 : count_total ... Pass
Test 4/7 : mean_bestSalesRank ... Pass
Test 5/7 : numNulls_bestSalesCategory ... Pass
Test 6/7 : numNulls_category ... Pass
Test 7/7 : variance_bestSalesRank ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 3





In [53]:
# %load -s task_3 assignment2.py
def task_3(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    asin_column = 'asin'
    price_column = 'price'
    attribute = 'also_viewed'
    related_column = 'related'
    # Outputs:
    meanPriceAlsoViewed_column = 'meanPriceAlsoViewed'
    countAlsoViewed_column = 'countAlsoViewed'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    
    sub_data = product_data[['asin', 'price']]
    
    # count viewed
    def get_length(x):
        if type(x) != dict or 'also_viewed' not in x:
            return None
        return len(x['also_viewed'])
    
    get_len = F.udf(get_length, T.IntegerType())
    df = product_data.select(F.col('asin'), get_len(F.col("related")).alias("countAlsoViewed"))
    
    mapping_data = product_data[['asin', 'price']]
    sub_data = product_data[['asin', 'related']]
    def get_lst(x):
        if type(x) != dict or 'also_viewed' not in x or x['also_viewed'] == []:
            return None
        return x['also_viewed']
    get_lst = F.udf(get_lst, T.ArrayType(T.StringType()))
    converted_df = sub_data.select(F.col('asin'), get_lst(F.col("related")).alias("extracted"))
    converted_df = converted_df.toDF('asin', 'extracted')#.show()
    exploded_df = converted_df.select(F.col('asin').alias('benchmark'), F.explode(F.col("extracted")).alias('asin'))
    merged = exploded_df.join(mapping_data, on='asin', how="left") # .groupBy('benchmark')
    transformed = merged[['benchmark', 'price']].groupBy('benchmark').mean().toDF('asin', 'meanPriceAlsoViewed')
    transformed = df.join(transformed, on='asin', how='left')
    



    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanPriceAlsoViewed': None,
        'variance_meanPriceAlsoViewed': None,
        'numNulls_meanPriceAlsoViewed': None,
        'mean_countAlsoViewed': None,
        'variance_countAlsoViewed': None,
        'numNulls_countAlsoViewed': None
    }
    # Modify res:
    res['count_total'] = transformed.count()
    res['mean_meanPriceAlsoViewed'] = transformed.select(F.avg(F.col('meanPriceAlsoViewed'))).head()[0]
    res['variance_meanPriceAlsoViewed'] = transformed.select(F.variance(F.col('meanPriceAlsoViewed'))).head()[0]
    res['numNulls_meanPriceAlsoViewed'] = transformed.select([F.count(F.when(F.col('meanPriceAlsoViewed').isNull(), 'meanPriceAlsoViewed')).alias('meanPriceAlsoViewed')]).head()[0]
    res['mean_countAlsoViewed'] = transformed.select(F.avg(F.col('countAlsoViewed'))).head()[0]
    res['variance_countAlsoViewed'] = transformed.select(F.variance(F.col('countAlsoViewed'))).head()[0]
    res['numNulls_countAlsoViewed'] = transformed.select([F.count(F.when(F.col('countAlsoViewed').isNull(), 'countAlsoViewed')).alias('countAlsoViewed')]).head()[0]



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_3')
    return res
    # -------------------------------------------------------------------------


In [54]:
res = task_3(data_io, data_dict['product'])
pa2.tests.test(res, 'task_3')

tests for task_3 --------------------------------------------------------------
Test 1/7 : count_total ... Pass
Test 2/7 : mean_countAlsoViewed ... Pass
Test 3/7 : mean_meanPriceAlsoViewed ... Pass
Test 4/7 : numNulls_countAlsoViewed ... Pass
Test 5/7 : numNulls_meanPriceAlsoViewed ... Pass
Test 6/7 : variance_countAlsoViewed ... Pass
Test 7/7 : variance_meanPriceAlsoViewed ... Pass
7/7 passed
-------------------------------------------------------------------------------


True

# Task 4

In [24]:
# %load -s task_4 assignment2.py
def task_4(data_io, product_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    price_column = 'price'
    title_column = 'title'
    # Outputs:
    meanImputedPrice_column = 'meanImputedPrice'
    medianImputedPrice_column = 'medianImputedPrice'
    unknownImputedTitle_column = 'unknownImputedTitle'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------

    imputer = K.Imputer()
    imputer.setInputCols(["price"])
    imputer.setOutputCols(["medianImputedPrice"])
    fit_median = imputer.setStrategy("median").fit(product_data[['price']]).transform(product_data[['price']])
    imputer.setOutputCols(["meanImputedPrice"])
    fit_mean = imputer.setStrategy("mean").fit(product_data[['price']]).transform(product_data[['price']])
    title_df = product_data.select(F.col('title')).fillna('unknown').withColumnRenamed('title', 'unknownImputedTitle')#.show()

    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'mean_meanImputedPrice': None,
        'variance_meanImputedPrice': None,
        'numNulls_meanImputedPrice': None,
        'mean_medianImputedPrice': None,
        'variance_medianImputedPrice': None,
        'numNulls_medianImputedPrice': None,
        'numUnknowns_unknownImputedTitle': None
    }
    # Modify res:
    res['count_total'] = title_df.count()
    res['mean_meanImputedPrice'] = fit_mean.select(F.avg(F.col('meanImputedPrice'))).head()[0]
    res['variance_meanImputedPrice'] = fit_mean.select(F.variance(F.col('meanImputedPrice'))).head()[0]
    res['numNulls_meanImputedPrice'] = fit_mean.select([F.count(F.when(F.col('meanImputedPrice').isNull(), 'meanImputedPrice')).alias('meanImputedPrice')]).head()[0]
    res['mean_medianImputedPrice'] = fit_median.select(F.avg(F.col('medianImputedPrice'))).head()[0]
    res['variance_medianImputedPrice'] = fit_median.select(F.variance(F.col('medianImputedPrice'))).head()[0]
    res['numNulls_medianImputedPrice'] = fit_median.select([F.count(F.when(F.col('medianImputedPrice').isNull(), 'medianImputedPrice')).alias('medianImputedPrice')]).head()[0]
    res['numUnknowns_unknownImputedTitle'] = title_df.filter(F.col("unknownImputedTitle") == "unknown").count()



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_4')
    return res
    # -------------------------------------------------------------------------


In [25]:
res = task_4(data_io, data_dict['product'])
pa2.tests.test(res, 'task_4')

tests for task_4 --------------------------------------------------------------
Test 1/8 : count_total ... Pass
Test 2/8 : mean_meanImputedPrice ... Pass
Test 3/8 : mean_medianImputedPrice ... Pass
Test 4/8 : numNulls_meanImputedPrice ... Pass
Test 5/8 : numNulls_medianImputedPrice ... Pass
Test 6/8 : numUnknowns_unknownImputedTitle ... Pass
Test 7/8 : variance_meanImputedPrice ... Pass
Test 8/8 : variance_medianImputedPrice ... Pass
8/8 passed
-------------------------------------------------------------------------------


True

# Task 5

In [6]:
# %load -s task_5 assignment2.py
def task_5(data_io, product_processed_data, word_0, word_1, word_2):
    # -----------------------------Column names--------------------------------
    # Inputs:
    title_column = 'title'
    # Outputs:
    titleArray_column = 'titleArray'
    titleVector_column = 'titleVector'
    # -------------------------------------------------------------------------

    # ---------------------- Your implementation begins------------------------
    array_df = product_processed_data.select(F.split(F.lower(product_processed_data['title']), ' ').alias('titleArray')) # .show()
    word2Vec = M.feature.Word2Vec(vectorSize = 16, minCount=100, seed=102, numPartitions = 4, inputCol = "titleArray", outputCol = "titleVector")
    model = word2Vec.fit(array_df)
    
    product_processed_data_output = model.transform(array_df)#.count()
#     size_vocabulary = model.getVectors().count()
#     word_0_synonyms = model.findSynonymsArray(word_0, 10)
#     word_1_synonyms = model.findSynonymsArray(word_1, 10)
#     word_2_synonyms = model.findSynonymsArray(word_2, 10)


    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'size_vocabulary': None,
        'word_0_synonyms': None,
        'word_1_synonyms': None,
        'word_2_synonyms': None
    }
    # Modify res:
    res['count_total'] = product_processed_data_output.count()
    res['size_vocabulary'] = model.getVectors().count()
    for name, word in zip(
        ['word_0_synonyms', 'word_1_synonyms', 'word_2_synonyms'],
        [word_0, word_1, word_2]
    ):
        res[name] = model.findSynonymsArray(word, 10)
    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_5')
    return res
    # -------------------------------------------------------------------------


In [7]:
res = task_5(data_io, data_dict['product_processed'], 'piano', 'rice', 'laptop')
pa2.tests.test(res, 'task_5')

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


tests for task_5 --------------------------------------------------------------
Test 1/8 : count_total ... Pass
Test 2/8 : size_vocabulary ... Pass
Test 3/8 : word_0_synonyms-length ... Pass
Test 4/8 : word_0_synonyms-correctness ... Pass
Test 5/8 : word_1_synonyms-length ... Pass
Test 6/8 : word_1_synonyms-correctness ... Pass
Test 7/8 : word_2_synonyms-length ... Pass
Test 8/8 : word_2_synonyms-correctness ... Pass
5/5 passed
-------------------------------------------------------------------------------


True

# Task 6

In [43]:
# %load -s task_6 assignment2.py
def task_6(data_io, product_processed_data):
    # -----------------------------Column names--------------------------------
    # Inputs:
    category_column = 'category'
    # Outputs:
    categoryIndex_column = 'categoryIndex'
    categoryOneHot_column = 'categoryOneHot'
    categoryPCA_column = 'categoryPCA'
    # -------------------------------------------------------------------------    

    # ---------------------- Your implementation begins------------------------
    
    sub = product_processed_data[[category_column]]
    string_indexer = K.StringIndexer(inputCol = category_column, outputCol = categoryIndex_column)
    string_indexer_model = string_indexer.fit(sub)
    indexed_df = string_indexer_model.transform(sub)#.show()
    encoder = K.OneHotEncoderEstimator(inputCols = [categoryIndex_column], outputCols = [categoryOneHot_column], dropLast=False)
    encoded_df = encoder.fit(indexed_df).transform(indexed_df)
    pca = K.PCA(k=15, inputCol=categoryOneHot_column, outputCol=categoryPCA_column)
    result_df = pca.fit(encoded_df).transform(encoded_df)



    # -------------------------------------------------------------------------

    # ---------------------- Put results in res dict --------------------------
    res = {
        'count_total': None,
        'meanVector_categoryOneHot': [None, ],
        'meanVector_categoryPCA': [None, ]
    }
    # Modify res:
    count_total = result_df.count()
    meanVector_categoryOneHot = result_df.select(M.stat.Summarizer.mean(result_df[categoryOneHot_column])).head()[0]
    meanVector_categoryPCA = result_df.select(M.stat.Summarizer.mean(result_df[categoryPCA_column])).head()[0]
    
    res['count_total'] = count_total
    res['meanVector_categoryOneHot'] = meanVector_categoryOneHot
    res['meanVector_categoryPCA'] = meanVector_categoryPCA



    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_6')
    return res
    # -------------------------------------------------------------------------


In [44]:
res = task_6(data_io, data_dict['product_processed'])
pa2.tests.test(res, 'task_6')

tests for task_6 --------------------------------------------------------------
Test 1/9 : count_total ... Pass
Test 2/9 : meanVector_categoryOneHot-length ... Pass
Test 3/9 : meanVector_categoryOneHot-sum ... Pass
Test 4/9 : meanVector_categoryOneHot-mean ... Pass
Test 5/9 : meanVector_categoryOneHot-variance ... Pass
Test 6/9 : meanVector_categoryPCA-length ... Pass
Test 7/9 : meanVector_categoryPCA-sum ... Pass
Test 8/9 : meanVector_categoryPCA-mean ... Pass
Test 9/9 : meanVector_categoryPCA-variance ... Pass
9/9 passed
-------------------------------------------------------------------------------


True

In [45]:
print ("End to end time: {}".format(time.time()-begin))

End to end time: 5728.93918299675


# Part 2: Model Selection

In [4]:
# Bring the part_2 datasets to memory and de-cache part_1 datasets.
# Execute this once before you start working on this Part
data_dict, _ = data_io.cache_switch(data_dict, 'part_2')

# Task 7

In [37]:
def task_7(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    
    rf = M.regression.RandomForestRegressor(maxDepth=5).setLabelCol("overall").setFeaturesCol("features")
    mdl = rf.fit(train_data)
    preds_df = mdl.transform(test_data)
    metrics = M.evaluation.RegressionEvaluator(predictionCol = "overall", labelCol = "prediction", metricName='rmse')
    test_rmse = metrics.evaluate(preds_df)
    
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None
    }
    # Modify res:
    res['test_rmse'] = test_rmse

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_7')
    return res
    # -------------------------------------------------------------------------

In [38]:
res = task_7(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_7')

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


tests for task_7 --------------------------------------------------------------
Test 1/1 : test_rmse ... Pass
1/1 passed
-------------------------------------------------------------------------------


True

# Task 8

In [17]:
def task_8(data_io, train_data, test_data):
    
    # ---------------------- Your implementation begins------------------------
    
    train, val = train_data.randomSplit(weights=[0.75, 0.25], seed=102)

    depths = [5, 7, 9, 12]
    candidates = []
    for depth in depths:
        rf = M.regression.RandomForestRegressor(maxDepth=depth).setLabelCol("overall").setFeaturesCol("features")
        mdl = rf.fit(train)
        preds_df = mdl.transform(val)
        metrics = M.evaluation.RegressionEvaluator(predictionCol = "overall", labelCol = "prediction", metricName='rmse')
        val_rmse = metrics.evaluate(preds_df)
        candidates.append((val_rmse, mdl))

    val_rmses = [i[0] for i in candidates]
    valid_rmse_depth_5 = val_rmses[0]
    valid_rmse_depth_7 = val_rmses[1]
    valid_rmse_depth_9 = val_rmses[2]
    valid_rmse_depth_12 = val_rmses[3]

    best_mdl = min(candidates, key=lambda x:x[0])[1]
    final_pred = best_mdl.transform(test_data)
    metrics = M.evaluation.RegressionEvaluator(predictionCol = "overall", labelCol = "prediction", metricName='rmse')
    test_rmse = metrics.evaluate(final_pred)
    
    
    
    # -------------------------------------------------------------------------
    
    
    # ---------------------- Put results in res dict --------------------------
    res = {
        'test_rmse': None,
        'valid_rmse_depth_5': None,
        'valid_rmse_depth_7': None,
        'valid_rmse_depth_9': None,
        'valid_rmse_depth_12': None,
    }
    # Modify res:
    res['test_rmse'] = test_rmse
    res['valid_rmse_depth_5'] = valid_rmse_depth_5
    res['valid_rmse_depth_7'] = valid_rmse_depth_7
    res['valid_rmse_depth_9'] = valid_rmse_depth_9
    res['valid_rmse_depth_12'] = valid_rmse_depth_12

    # -------------------------------------------------------------------------

    # ----------------------------- Do not change -----------------------------
    data_io.save(res, 'task_8')
    return res
    # -------------------------------------------------------------------------

In [19]:
res = task_8(data_io, data_dict['ml_features_train'], data_dict['ml_features_test'])
pa2.tests.test(res, 'task_8')

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


tests for task_8 --------------------------------------------------------------
Test 1/5 : test_rmse ... Pass
Test 2/5 : valid_rmse_depth_12 ... Pass
Test 3/5 : valid_rmse_depth_5 ... Pass
Test 4/5 : valid_rmse_depth_7 ... Pass
Test 5/5 : valid_rmse_depth_9 ... Pass
5/5 passed
-------------------------------------------------------------------------------


True

In [20]:
print ("End to end time: {}".format(time.time()-begin))

End to end time: 1528.9998021125793
